In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats 
import os
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.metrics import r2_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import datetime
import random

In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df_item = pd.read_csv("olist_order_items_dataset.csv") ## used
df_item.dropna()
df_reviews = pd.read_csv("olist_order_reviews_dataset.csv") ## used
df_reviews.dropna()
df_orders = pd.read_csv("olist_orders_dataset.csv")
df_orders.dropna()
df_products = pd.read_csv("olist_products_dataset.csv") ## used
df_products.dropna()
df_geolocation = pd.read_csv("olist_geolocation_dataset.csv")
df_geolocation.dropna()
df_sellers = pd.read_csv("olist_sellers_dataset.csv") ## used
df_sellers.dropna()
df_order_pay = pd.read_csv("olist_order_payments_dataset.csv") ## used
df_order_pay.dropna()
df_customers = pd.read_csv("olist_customers_dataset.csv") ## used
df_customers.dropna()
df_category = pd.read_csv("olist_product_category_name_translation.csv") ## used
df_category.dropna()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor
...,...,...
66,flores,flowers
67,artes_e_artesanato,arts_and_craftmanship
68,fraldas_higiene,diapers_and_hygiene
69,fashion_roupa_infanto_juvenil,fashion_childrens_clothes


In [2]:
df_train = df_orders.merge(df_item, on='order_id', how='left')
df_train = df_train.merge(df_order_pay, on='order_id', how='outer', validate='m:m')
df_train = df_train.merge(df_reviews, on='order_id', how='outer')
df_train = df_train.merge(df_products, on='product_id', how='outer')
df_train = df_train.merge(df_customers, on='customer_id', how='outer')
df_train = df_train.merge(df_sellers, on='seller_id', how='outer')
df_train = df_train.merge(df_category, on='product_category_name', how='outer')


In [3]:
df_train['order_item_id'].unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., nan])

In [4]:
df_train.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'review_id', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'seller_zip_code_prefix', 'seller_city',
       'seller_state', 'product_category_name_english'],
      dtype='object')

In [5]:
df_train = df_train.drop(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
                          'order_approved_at', 'order_item_id', 'product_id', 'seller_id',
                         'payment_value', 'review_id', 'review_comment_title', 'review_comment_message',
                         'product_name_lenght', 'product_weight_g', 'product_length_cm', 'product_height_cm',
                         'product_width_cm', 'customer_unique_id', 'customer_zip_code_prefix', 'seller_zip_code_prefix',
                         'seller_city', 'seller_state', 'product_category_name', 'review_answer_timestamp'], axis=1)

In [6]:
df_train.head()

,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,review_score,review_creation_date,product_description_lenght,product_photos_qty,customer_city,customer_state,product_category_name_english
0,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2017-10-06 11:07:15,29.99,8.72,1.0,credit_card,1.0,4,2017-10-11 00:00:00,268.0,4.0,sao paulo,SP,housewares
1,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2017-10-06 11:07:15,29.99,8.72,3.0,voucher,1.0,4,2017-10-11 00:00:00,268.0,4.0,sao paulo,SP,housewares
2,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2017-10-06 11:07:15,29.99,8.72,2.0,voucher,1.0,4,2017-10-11 00:00:00,268.0,4.0,sao paulo,SP,housewares
3,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28 00:00:00,2017-08-21 20:05:16,29.99,7.78,1.0,credit_card,3.0,4,2017-08-19 00:00:00,268.0,4.0,sao paulo,SP,housewares
4,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15 00:00:00,2017-08-08 18:37:31,29.99,7.78,1.0,credit_card,1.0,5,2017-08-08 00:00:00,268.0,4.0,sao paulo,SP,housewares


In [7]:
print(df_train.dtypes)

order_delivered_carrier_date      object
order_delivered_customer_date     object
order_estimated_delivery_date     object
shipping_limit_date               object
price                            float64
freight_value                    float64
payment_sequential               float64
payment_type                      object
payment_installments             float64
review_score                       int64
review_creation_date              object
product_description_lenght       float64
product_photos_qty               float64
customer_city                     object
customer_state                    object
product_category_name_english     object
dtype: object


In [8]:
#df_train['order_delivered_carrier_date'].astype(dt)
df_train['order_delivered_carrier_date'] =  pd.to_datetime(df_train['order_delivered_carrier_date'], infer_datetime_format=True)
df_train['order_delivered_customer_date'] =  pd.to_datetime(df_train['order_delivered_customer_date'], infer_datetime_format=True)
df_train['order_estimated_delivery_date'] =  pd.to_datetime(df_train['order_estimated_delivery_date'], infer_datetime_format=True)
df_train['shipping_limit_date'] =  pd.to_datetime(df_train['shipping_limit_date'], infer_datetime_format=True)
df_train['review_creation_date'] =  pd.to_datetime(df_train['review_creation_date'], infer_datetime_format=True)

print(df_train.dtypes)

order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
shipping_limit_date              datetime64[ns]
price                                   float64
freight_value                           float64
payment_sequential                      float64
payment_type                             object
payment_installments                    float64
review_score                              int64
review_creation_date             datetime64[ns]
product_description_lenght              float64
product_photos_qty                      float64
customer_city                            object
customer_state                           object
product_category_name_english            object
dtype: object


In [16]:
print(df_train.dtypes)

order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
shipping_limit_date              datetime64[ns]
price                                   float64
freight_value                           float64
payment_sequential                      float64
payment_type                             object
payment_installments                    float64
review_score                              int64
review_creation_date             datetime64[ns]
product_description_lenght              float64
product_photos_qty                      float64
customer_city                            object
customer_state                           object
product_category_name_english            object
Difference                               string
dtype: object


In [21]:
df_train = df_train.dropna()

In [23]:
df_train['Difference'] = df_train['order_estimated_delivery_date'].sub(df_train['order_delivered_customer_date'], axis=0)
df_train['Difference'] = df_train['Difference'].astype('string')
df_train['Difference'] = df_train['Difference'].str.split('d').str[0]
df_train['Difference'] = df_train['Difference'].astype('int')

df_train.head(3) 


,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,review_score,review_creation_date,product_description_lenght,product_photos_qty,customer_city,customer_state,product_category_name_english,Difference
0,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,2017-10-06 11:07:15,29.99,8.72,1.0,credit_card,1.0,4,2017-10-11,268.0,4.0,sao paulo,SP,housewares,7
1,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,2017-10-06 11:07:15,29.99,8.72,3.0,voucher,1.0,4,2017-10-11,268.0,4.0,sao paulo,SP,housewares,7
2,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,2017-10-06 11:07:15,29.99,8.72,2.0,voucher,1.0,4,2017-10-11,268.0,4.0,sao paulo,SP,housewares,7


In [31]:
def is_late_func(x):
    if x>0:
        return 0
    if x<0:
        return 1
    
df_train['Is It Late?'] = df_train['Difference'].apply(is_late_func)

In [32]:
#1=Late 0=OnTime

df_train = df_train.dropna()
df_train['Is It Late?'] = df_train['Is It Late?'].astype('int')
print(df_train.dtypes)
df_train.head(30)


order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
shipping_limit_date              datetime64[ns]
price                                   float64
freight_value                           float64
payment_sequential                      float64
payment_type                             object
payment_installments                    float64
review_score                              int64
review_creation_date             datetime64[ns]
product_description_lenght              float64
product_photos_qty                      float64
customer_city                            object
customer_state                           object
product_category_name_english            object
Difference                                int64
Is It Late?                               int64
dtype: object


,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,review_score,review_creation_date,product_description_lenght,product_photos_qty,customer_city,customer_state,product_category_name_english,Difference,Is It Late?
0,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,2017-10-06 11:07:15,29.99,8.72,1.0,credit_card,1.0,4,2017-10-11,268.0,4.0,sao paulo,SP,housewares,7,0
1,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,2017-10-06 11:07:15,29.99,8.72,3.0,voucher,1.0,4,2017-10-11,268.0,4.0,sao paulo,SP,housewares,7,0
2,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,2017-10-06 11:07:15,29.99,8.72,2.0,voucher,1.0,4,2017-10-11,268.0,4.0,sao paulo,SP,housewares,7,0
3,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28,2017-08-21 20:05:16,29.99,7.78,1.0,credit_card,3.0,4,2017-08-19,268.0,4.0,sao paulo,SP,housewares,9,0
4,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15,2017-08-08 18:37:31,29.99,7.78,1.0,credit_card,1.0,5,2017-08-08,268.0,4.0,sao paulo,SP,housewares,7,0
5,2017-10-27 16:48:46,2017-11-07 18:04:59,2017-11-13,2017-10-31 02:14:11,29.99,14.10,1.0,boleto,1.0,3,2017-11-08,268.0,4.0,florianopolis,SC,housewares,5,0
6,2017-08-25 14:17:55,2017-08-28 20:12:20,2017-09-14,2017-08-30 11:45:25,46.80,67.70,1.0,credit_card,1.0,1,2017-08-29,189.0,3.0,sao paulo,SP,housewares,16,0
7,2017-09-26 17:27:54,2017-10-19 21:09:21,2017-11-13,2017-09-27 22:25:11,28.90,21.15,1.0,credit_card,1.0,4,2017-10-20,322.0,5.0,manaus,AM,housewares,24,0
8,2017-08-18 15:41:40,2017-08-21 15:05:13,2017-08-29,2017-08-22 14:55:20,24.90,7.78,1.0,credit_card,1.0,5,2017-08-22,749.0,4.0,sao paulo,SP,housewares,7,0
9,2018-07-30 09:15:00,2018-08-02 13:45:01,2018-08-20,2018-07-31 22:24:21,64.99,23.26,1.0,credit_card,2.0,3,2018-08-03,220.0,1.0,sombrio,SC,housewares,17,0


In [33]:
import datetime as dt

#Monday is 0 Sunday is 6

df_train['Date Given to Carrier'] = df_train['order_delivered_carrier_date'].dt.dayofweek
df_train['Date Delivered to Customer'] = df_train['order_delivered_customer_date'].dt.dayofweek
df_train['Estimated Delivery Date'] = df_train['order_estimated_delivery_date'].dt.dayofweek
df_train['Shipping Limit Date'] = df_train['shipping_limit_date'].dt.dayofweek
df_train['Review Creation Date'] = df_train['review_creation_date'].dt.dayofweek


df_train = df_train.drop(['order_delivered_carrier_date'], axis=1)
df_train = df_train.drop(['order_delivered_customer_date'], axis=1)
df_train = df_train.drop(['order_estimated_delivery_date'], axis=1)
df_train = df_train.drop(['shipping_limit_date'], axis=1)
df_train = df_train.drop(['review_creation_date'], axis=1)


In [34]:
df_train['payment_type'] = df_train['payment_type'].astype('string')
print(df_train.dtypes)

price                            float64
freight_value                    float64
payment_sequential               float64
payment_type                      string
payment_installments             float64
review_score                       int64
product_description_lenght       float64
product_photos_qty               float64
customer_city                     object
customer_state                    object
product_category_name_english     object
Difference                         int64
Is It Late?                        int64
Date Given to Carrier              int64
Date Delivered to Customer         int64
Estimated Delivery Date            int64
Shipping Limit Date                int64
Review Creation Date               int64
dtype: object


What factor has the most impact on customer satisfaction?

1. Are people who place large orders more likely to leave bad review?
2. Does customer location have an impact on their review?
3. Should we remind customers to leave a review on weekends or weekdays?
4. Are late shipments more likely to leave bad reviews?
5. Does payment type affect customer satisfaction?

In [35]:
from sklearn.preprocessing import LabelEncoder

#Convert payment type from object to string
df_train['payment_type'] = df_train['payment_type'].astype('string')
df_train['product_category_name_english'] = df_train['product_category_name_english'].astype('string')

print(df_train.dtypes)

price                            float64
freight_value                    float64
payment_sequential               float64
payment_type                      string
payment_installments             float64
review_score                       int64
product_description_lenght       float64
product_photos_qty               float64
customer_city                     object
customer_state                    object
product_category_name_english     string
Difference                         int64
Is It Late?                        int64
Date Given to Carrier              int64
Date Delivered to Customer         int64
Estimated Delivery Date            int64
Shipping Limit Date                int64
Review Creation Date               int64
dtype: object


In [36]:
#1=Credit Card, #5=Voucher, #0=Boleto, #2=Debit Card, #3=NAN, #4=Not Defined

lb_make = LabelEncoder()
df_train["Payment Type"] = lb_make.fit_transform(df_train["payment_type"])
df_train["Customer City"] = lb_make.fit_transform(df_train["customer_city"])
df_train["Customer State"] = lb_make.fit_transform(df_train["customer_state"])
df_train["Product Category Name"] = lb_make.fit_transform(df_train["product_category_name_english"])

df_train

,price,freight_value,payment_sequential,payment_type,payment_installments,review_score,product_description_lenght,product_photos_qty,customer_city,customer_state,...,Is It Late?,Date Given to Carrier,Date Delivered to Customer,Estimated Delivery Date,Shipping Limit Date,Review Creation Date,Payment Type,Customer City,Customer State,Product Category Name
0,29.99,8.72,1.0,credit_card,1.0,4,268.0,4.0,sao paulo,SP,...,0,2,1,2,4,2,1,3541,25,49
1,29.99,8.72,3.0,voucher,1.0,4,268.0,4.0,sao paulo,SP,...,0,2,1,2,4,2,3,3541,25,49
2,29.99,8.72,2.0,voucher,1.0,4,268.0,4.0,sao paulo,SP,...,0,2,1,2,4,2,3,3541,25,49
3,29.99,7.78,1.0,credit_card,3.0,4,268.0,4.0,sao paulo,SP,...,0,3,4,0,0,5,1,3541,25,49
4,29.99,7.78,1.0,credit_card,1.0,5,268.0,4.0,sao paulo,SP,...,0,4,0,1,1,1,1,3541,25,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119144,64.99,19.37,1.0,credit_card,1.0,5,345.0,1.0,belem,PA,...,0,1,1,3,4,2,1,440,13,9
119145,44.99,18.42,1.0,boleto,1.0,1,568.0,1.0,paranagua,PR,...,0,4,0,4,2,1,0,2647,17,9
119147,58.99,7.39,1.0,credit_card,1.0,5,459.0,1.0,ribeirao pires,SP,...,0,4,0,0,1,2,1,3082,25,9
119148,23.99,7.43,1.0,boleto,1.0,5,706.0,1.0,sao paulo,SP,...,0,4,5,3,1,6,0,3541,25,9


In [37]:
df_train = df_train.drop(['payment_type'], axis=1)
df_train = df_train.drop(['customer_city'], axis=1)
df_train = df_train.drop(['customer_state'], axis=1)
df_train = df_train.drop(['product_category_name_english'], axis=1)

In [38]:
df_train.head()

,price,freight_value,payment_sequential,payment_installments,review_score,product_description_lenght,product_photos_qty,Difference,Is It Late?,Date Given to Carrier,Date Delivered to Customer,Estimated Delivery Date,Shipping Limit Date,Review Creation Date,Payment Type,Customer City,Customer State,Product Category Name
0,29.99,8.72,1.0,1.0,4,268.0,4.0,7,0,2,1,2,4,2,1,3541,25,49
1,29.99,8.72,3.0,1.0,4,268.0,4.0,7,0,2,1,2,4,2,3,3541,25,49
2,29.99,8.72,2.0,1.0,4,268.0,4.0,7,0,2,1,2,4,2,3,3541,25,49
3,29.99,7.78,1.0,3.0,4,268.0,4.0,9,0,3,4,0,0,5,1,3541,25,49
4,29.99,7.78,1.0,1.0,5,268.0,4.0,7,0,4,0,1,1,1,1,3541,25,49


In [39]:
df_train['payment_sequential'].unique()

array([ 1.,  3.,  2.,  5.,  6.,  4.,  8.,  9.,  7., 13., 11., 12., 10.,
       17., 16., 19., 21., 18., 15., 20., 14., 22., 26., 25., 24., 23.])

In [40]:

df_train['payment_sequential'] = df_train['payment_sequential'].astype('int')
df_train['payment_installments'] = df_train['payment_installments'].astype('int')
df_train['product_description_lenght'] = df_train['product_description_lenght'].astype('int')
df_train['product_photos_qty'] = df_train['product_photos_qty'].astype('int')
df_train['Date Given to Carrier'] = df_train['Date Given to Carrier'].astype('int')
df_train['Date Delivered to Customer'] = df_train['Date Delivered to Customer'].astype('int')
df_train['Shipping Limit Date'] = df_train['Shipping Limit Date'].astype('int')

In [41]:
print(df_train.dtypes)

price                         float64
freight_value                 float64
payment_sequential              int64
payment_installments            int64
review_score                    int64
product_description_lenght      int64
product_photos_qty              int64
Difference                      int64
Is It Late?                     int64
Date Given to Carrier           int64
Date Delivered to Customer      int64
Estimated Delivery Date         int64
Shipping Limit Date             int64
Review Creation Date            int64
Payment Type                    int64
Customer City                   int64
Customer State                  int64
Product Category Name           int64
dtype: object


In [42]:
df_train

,price,freight_value,payment_sequential,payment_installments,review_score,product_description_lenght,product_photos_qty,Difference,Is It Late?,Date Given to Carrier,Date Delivered to Customer,Estimated Delivery Date,Shipping Limit Date,Review Creation Date,Payment Type,Customer City,Customer State,Product Category Name
0,29.99,8.72,1,1,4,268,4,7,0,2,1,2,4,2,1,3541,25,49
1,29.99,8.72,3,1,4,268,4,7,0,2,1,2,4,2,3,3541,25,49
2,29.99,8.72,2,1,4,268,4,7,0,2,1,2,4,2,3,3541,25,49
3,29.99,7.78,1,3,4,268,4,9,0,3,4,0,0,5,1,3541,25,49
4,29.99,7.78,1,1,5,268,4,7,0,4,0,1,1,1,1,3541,25,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119144,64.99,19.37,1,1,5,345,1,22,0,1,1,3,4,2,1,440,13,9
119145,44.99,18.42,1,1,1,568,1,17,0,4,0,4,2,1,0,2647,17,9
119147,58.99,7.39,1,1,5,459,1,6,0,4,0,0,1,2,1,3082,25,9
119148,23.99,7.43,1,1,5,706,1,11,0,4,5,3,1,6,0,3541,25,9


In [48]:
#0 = Positive #1 = Negative
def is_late_func(x):
    if x>3:
        return 0
    else:
        return 1
    
df_train['Positive or Negative'] = df_train['review_score'].apply(is_late_func)

df_train.head(2)

,price,freight_value,payment_sequential,payment_installments,review_score,product_description_lenght,product_photos_qty,Difference,Is It Late?,Date Given to Carrier,Date Delivered to Customer,Estimated Delivery Date,Shipping Limit Date,Review Creation Date,Payment Type,Customer City,Customer State,Product Category Name,Positive or Negative
0,29.99,8.72,1,1,4,268,4,7,0,2,1,2,4,2,1,3541,25,49,0
1,29.99,8.72,3,1,4,268,4,7,0,2,1,2,4,2,3,3541,25,49,0


In [49]:
df_train = df_train.dropna()
df_train['Is It Late?'] = df_train['Is It Late?'].astype('int')
print(df_train.dtypes)


price                         float64
freight_value                 float64
payment_sequential              int64
payment_installments            int64
review_score                    int64
product_description_lenght      int64
product_photos_qty              int64
Difference                      int64
Is It Late?                     int64
Date Given to Carrier           int64
Date Delivered to Customer      int64
Estimated Delivery Date         int64
Shipping Limit Date             int64
Review Creation Date            int64
Payment Type                    int64
Customer City                   int64
Customer State                  int64
Product Category Name           int64
Positive or Negative            int64
dtype: object


In [50]:
df_train['Total Order Value'] = df_train['price'] + df_train['freight_value']

In [59]:
df_train = df_train.drop(['review_score', 'Difference', 'payment_sequential', 'payment_installments', 'Date Given to Carrier',
               'Estimated Delivery Date', 'Shipping Limit Date', 'price', 'freight_value'], axis=1)

In [60]:
target = df_train["Positive or Negative"]
target_names = ["negative", "positive"]

In [62]:
data = df_train.drop("Positive or Negative", axis=1)
feature_names = data.columns
data.head()

,product_description_lenght,product_photos_qty,Is It Late?,Date Delivered to Customer,Review Creation Date,Payment Type,Customer City,Customer State,Product Category Name,Total Order Value
0,268,4,0,1,2,1,3541,25,49,38.71
1,268,4,0,1,2,3,3541,25,49,38.71
2,268,4,0,1,2,3,3541,25,49,38.71
3,268,4,0,4,5,1,3541,25,49,37.77
4,268,4,0,0,1,1,3541,25,49,37.77


In [66]:
from sklearn.model_selection import train_test_split
from sklearn import tree

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [67]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7454636020778481

In [72]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.8384330747883014

In [73]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.22364186213563836, 'Total Order Value'),
 (0.19710932347035517, 'product_description_lenght'),
 (0.16140184211986222, 'Customer City'),
 (0.08999276028788349, 'Product Category Name'),
 (0.08106999160500782, 'Is It Late?'),
 (0.060136357432231495, 'Customer State'),
 (0.052799849855401815, 'Date Delivered to Customer'),
 (0.05064914327898172, 'Review Creation Date'),
 (0.04894758092929676, 'product_photos_qty'),
 (0.0342512888853412, 'Payment Type')]

In [74]:
corr_matrix = feat_eng.corr()
corr_matrix['review_score'].sort_values(ascending=False)

NameError: name 'feat_eng' is not defined